In [3]:
import os
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
import numpy as np
from sklearn import metrics
from xgboost.sklearn import XGBClassifier 
from sklearn.neural_network import MLPClassifier


In [7]:
input_path = '/app/liucd/判定_fill_df.csv'
rad_path = '/app/liucd/5center-1310-all+6status-pcr.csv'

In [21]:
rad_features = ['patient_id', 'dce-delta-log-sigma-5-0-mm-3D_glcm_DifferenceEntropy',
 'dce-bf-log-sigma-3-0-mm-3D_firstorder_Skewness',
 'dce-bf-log-sigma-4-0-mm-3D_gldm_DependenceEntropy',
 'dce-bf-log-sigma-5-0-mm-3D_firstorder_Skewness',
 'dce-bf-wavelet-LHL_firstorder_Mean',
 'dce-bf-wavelet-LHL_glcm_DifferenceEntropy',
 'dce-bf-wavelet-LHL_glrlm_RunEntropy',
 'dce-bf-wavelet-HLL_glszm_ZoneEntropy',
 'adc-bl-wavelet-LLL_gldm_SmallDependenceEmphasis']

In [24]:
def merge_df(df_raw_path):
    """
    输入原始的文件路径：没有delta的
    输出 既有原有的，又有delta的列
    """
    df_raw = pd.read_csv(df_raw_path)
    print(len(df_raw))
    cols = df_raw.columns[1: -1]

    col_bl = [name for name in cols if '-bl-' in name]
    col_bf = [name for name in cols if '-bf-' in name]
    print(len(col_bl), len(col_bf))
    df_first = df_raw[col_bl]
    df_second = df_raw[col_bf]
    df_first.columns = [name.replace('-bl-', '-delta-') for name in df_first.columns]
    df_second.columns = [name.replace('-bf-', '-delta-') for name in df_second.columns]

    df_delta = df_second - df_first  # 两个dataframe相减需要相同的行名和列名

    df_delta.insert(loc= 0 , column= 'patient_id', value= df_raw['patient_id'])

    df_merge = pd.merge(df_delta, df_raw, left_on='patient_id', right_on='patient_id')  # 方法2
    df_merge = pd.concat([df_delta, df_raw[df_raw.columns[1: ]]], axis=1)

    return df_merge

rad_df = merge_df(rad_path)
rad_df = rad_df[rad_features]
rad_df

1317
2264 2263


,patient_id,dce-delta-log-sigma-5-0-mm-3D_glcm_DifferenceEntropy,dce-bf-log-sigma-3-0-mm-3D_firstorder_Skewness,dce-bf-log-sigma-4-0-mm-3D_gldm_DependenceEntropy,dce-bf-log-sigma-5-0-mm-3D_firstorder_Skewness,dce-bf-wavelet-LHL_firstorder_Mean,dce-bf-wavelet-LHL_glcm_DifferenceEntropy,dce-bf-wavelet-LHL_glrlm_RunEntropy,dce-bf-wavelet-HLL_glszm_ZoneEntropy,adc-bl-wavelet-LLL_gldm_SmallDependenceEmphasis
0,976497,0.019366,0.219314,9.036620,0.161089,-1.015850,3.653617,4.870243,7.230573,0.651705
1,976855,-0.588563,-1.679852,7.960455,-1.592570,0.203749,3.318455,4.639385,7.042979,0.785428
2,978509,-0.082875,-0.478516,8.776517,-0.872217,-2.779912,3.168494,4.654114,7.323049,0.709319
3,1001769,-1.203932,-0.840375,7.938833,-0.618427,0.271760,2.916017,4.251457,7.056933,0.805726
4,1006675,-0.413856,-0.623015,8.560586,-0.665192,-0.093608,2.664824,4.230114,7.189380,0.686057
...,...,...,...,...,...,...,...,...,...,...
1312,576195,-0.931867,-1.136835,6.306491,-0.603338,0.079689,1.208392,3.411577,5.061482,0.666919
1313,574715,-0.356469,-1.350703,7.191078,-1.005959,-3.140468,1.668230,3.711391,6.210419,0.698343
1314,576570,-0.938677,-0.525061,6.411881,-0.302395,-0.050769,1.322478,3.650255,6.541261,0.723284
1315,619534,-1.194156,0.908799,7.717673,0.639229,-0.732751,3.124309,4.337691,6.747374,0.201321


In [28]:
cli_features = ['patient_ID', 'T_stage', 'HER2_status', 'NAC_classification', 'ER_percentage', 'PR_percentage', 'Ki_67']

clinical_df = pd.read_csv(input_path)  # 以第二行（header=1)作为标签，并删除前面的若干无效行

cln_df = clinical_df[cli_features + ['bpCR']]
cln_df.rename(columns={'patient_ID': 'patient_id'}, inplace=True)

cln_df

/tmp/ipykernel_1074266/536590301.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cln_df.rename(columns={'patient_ID': 'patient_id'}, inplace=True)


,patient_id,T_stage,HER2_status,NAC_classification,ER_percentage,PR_percentage,Ki_67,bpCR
0,966939,4,2,2,0.15,0.0,0.40,0
1,976497,4,0,2,0.15,0.0,0.05,0
2,976855,4,0,2,0.15,0.0,0.20,0
3,978509,4,0,2,0.70,0.9,0.10,0
4,1001769,2,1,2,0.50,0.1,0.15,0
...,...,...,...,...,...,...,...,...
1700,1565305,4,2,1,0.00,0.0,0.15,1
1701,1726198,2,1,2,0.90,0.8,0.20,0
1702,1550326,2,2,1,0.00,0.0,0.80,1
1703,1636744,3,2,1,0.00,0.0,0.60,1


In [32]:
merge_df = rad_df.merge(cln_df, on='patient_id')

In [33]:
merge_df[merge_df['patient_id']]

,patient_id,dce-delta-log-sigma-5-0-mm-3D_glcm_DifferenceEntropy,dce-bf-log-sigma-3-0-mm-3D_firstorder_Skewness,dce-bf-log-sigma-4-0-mm-3D_gldm_DependenceEntropy,dce-bf-log-sigma-5-0-mm-3D_firstorder_Skewness,dce-bf-wavelet-LHL_firstorder_Mean,dce-bf-wavelet-LHL_glcm_DifferenceEntropy,dce-bf-wavelet-LHL_glrlm_RunEntropy,dce-bf-wavelet-HLL_glszm_ZoneEntropy,adc-bl-wavelet-LLL_gldm_SmallDependenceEmphasis,T_stage,HER2_status,NAC_classification,ER_percentage,PR_percentage,Ki_67,bpCR
0,976497,0.019366,0.219314,9.036620,0.161089,-1.015850,3.653617,4.870243,7.230573,0.651705,4,0,2,0.15,0.00,0.05,0
1,976855,-0.588563,-1.679852,7.960455,-1.592570,0.203749,3.318455,4.639385,7.042979,0.785428,4,0,2,0.15,0.00,0.20,0
2,978509,-0.082875,-0.478516,8.776517,-0.872217,-2.779912,3.168494,4.654114,7.323049,0.709319,4,0,2,0.70,0.90,0.10,0
3,1001769,-1.203932,-0.840375,7.938833,-0.618427,0.271760,2.916017,4.251457,7.056933,0.805726,2,1,2,0.50,0.10,0.15,0
4,1006675,-0.413856,-0.623015,8.560586,-0.665192,-0.093608,2.664824,4.230114,7.189380,0.686057,4,1,2,0.15,0.00,0.30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,576195,-0.931867,-1.136835,6.306491,-0.603338,0.079689,1.208392,3.411577,5.061482,0.666919,2,0,2,0.15,0.00,0.80,1
1313,574715,-0.356469,-1.350703,7.191078,-1.005959,-3.140468,1.668230,3.711391,6.210419,0.698343,2,0,2,0.90,0.10,0.70,0
1314,576570,-0.938677,-0.525061,6.411881,-0.302395,-0.050769,1.322478,3.650255,6.541261,0.723284,3,2,2,0.35,0.01,0.20,1
1315,619534,-1.194156,0.908799,7.717673,0.639229,-0.732751,3.124309,4.337691,6.747374,0.201321,1,2,1,0.00,0.00,0.70,1
